## Imports

In [ ]:
import pandas as pd
import numpy as np
import calendar
import glob

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='spherical-berm-323321') # <-- Edit project as necessary

# Consistent Parameters
final = ee.FeatureCollection('users/andyc97/model_preprocessed/final_shapefile') # GEE asset location of shapefile which you want to iterate over
quarter_degree_grid = ee.FeatureCollection('users/andyc97/model_preprocessed/quarter_degree_grid') # GEE asset location where you want to save grid

## Make grid to iterate over

In [ ]:
# Define your FeatureCollection
boreal_tundra_region = final

# Define the spatial resolution and grid size in pixels.
resolution = 27830
grid_size_pixels = 64
grid_size_meters = resolution * grid_size_pixels

# Get the bounding box of the region
region_bounds = boreal_tundra_region.geometry().bounds()

# Get the coordinates of the bounding box
region_coords = region_bounds.coordinates().get(0).getInfo()

# Define the coordinates of the bounding box
min_lon = region_coords[0][0]
min_lat = region_coords[0][1]
max_lon = region_coords[2][0]
max_lat = region_coords[2][1]

# Create a list to hold the grid cells
grid_cells = []

# Generate the grid cells
lon = min_lon
while lon < max_lon:
    lat = min_lat
    while lat < max_lat:
        cell = ee.Geometry.Rectangle([lon, lat, lon + (grid_size_meters / 111320), lat + (grid_size_meters / 111320)])
        grid_cells.append(ee.Feature(cell))
        lat += (grid_size_meters / 111320)
    lon += (grid_size_meters / 111320)

# Create a FeatureCollection from the grid cells
grid_feature_collection = ee.FeatureCollection(grid_cells)

# Filter the grid cells to retain only those within the region
filtered_grid_cells = grid_feature_collection.filterBounds(boreal_tundra_region.geometry())
final_grid = ee.FeatureCollection(filtered_grid_cells)

# Print the number of grid cells created
print(f"Number of grid cells created: {final_grid.size().getInfo()}")

In [ ]:
# Export grid to asset
task = ee.batch.Export.table.toAsset(
    collection=ee.FeatureCollection(final_grid),
    description='test_grid',
    assetId='users/andyc97/model_preprocessed/quarter_degree_grid') # <-- Edit save location
task.start()

## Process CEMS to CSV

In [ ]:
# Import downscaled FWI into an image collection
# Run and save single years before combining into one master file
years = list(range(2014, 2015))  # 2001 to 2014
months = list(range(1, 13))  # 1 to 12

# Google Cloud Storage bucket path template to CEMS - DO NOT CHANGE!
gcs_template = 'gs://ce-cems-fire-daily-4-1/{year}{month}{day}.tif'

# Function to generate monthly mean image for a given year and month
def create_monthly_mean_image(year, month):
    # Get the number of days in the month
    _, last_day = calendar.monthrange(year, month)
    
    # Initialize an empty ImageCollection for the month
    monthly_images = ee.ImageCollection([])

    # Loop through each day of the month
    for day in range(1, last_day + 1):
        # Format the file path
        file_path = gcs_template.format(year=year, month=f'{month:02d}', day=f'{day:02d}')
        
        try:
            # Load the GeoTIFF image from GCS
            daily_image = ee.Image.loadGeoTIFF(file_path).clip(final)
                  
            # Handle missing pixels for BUI and FWI bands
            bui_filled = daily_image.select('build_up_index').unmask(-9999, sameFootprint=True).updateMask(daily_image.select('fine_fuel_moisture_code'))
            fwi_filled = daily_image.select('fire_weather_index').unmask(-9999, sameFootprint=True).updateMask(daily_image.select('fine_fuel_moisture_code'))
            
            # Replace the original BUI and FWI with the filled versions
            daily_image = daily_image.addBands([bui_filled.rename('build_up_index'), fwi_filled.rename('fire_weather_index')], overwrite=True)
            
            # Add the daily image to the collection
            monthly_images = monthly_images.merge(ee.ImageCollection([daily_image]))
        
        except Exception as e:
            # Handle missing files or errors in loading
            print(f'Failed to load {file_path}: {e}')
            continue

    # Calculate the mean of the monthly images
    monthly_mean_image = monthly_images.mean()
    
    # Get coordinates and add to the image
    coords = ee.Image.pixelLonLat().clip(final).updateMask(monthly_mean_image.select('fine_fuel_moisture_code'))
    monthly_mean_image = monthly_mean_image.addBands(coords)

    # Set properties including the start and end dates
    start_date = f'{year}-{month:02d}-01'
    end_date = f'{year}-{month:02d}-{last_day:02d}'
    
    monthly_mean_image = monthly_mean_image.set({
        'scenario': 'historical',
        'year': year,
        'month': month,
        'system:time_start': ee.Date(start_date).millis(),
        'system:time_end': ee.Date(end_date).millis()
    })

    return monthly_mean_image

# Create an empty list to hold the images
image_list = []

# Loop through scenarios, years, and months, and import the files
for year in years:
    for month in months:
        try:
            image = create_monthly_mean_image(year, month)
            image_list.append(image)
        except Exception as e:
            print(f"Error loading image for {year}-{month}: {e}")

# Convert the list of images to an Earth Engine ImageCollection
monthly_mean_collection = ee.ImageCollection(image_list)

# Print the image collection size to confirm
print("Image collection created with", monthly_mean_collection.size().getInfo(), "images")

In [ ]:
# Iterate over images
image_collection = monthly_mean_collection

# Function to extract data for all grid cells from each image in the ImageCollection
def extract_data_for_all_cells(image):
    # Use reduceRegions to process all grid cells at once, instead of iterating over them
    data = image.reduceRegions(
        collection=quarter_degree_grid,
        reducer=ee.Reducer.toList(),
        scale=27829.87269831839,
        crs='EPSG:4326'
    )
    return data

# Iterate over images and extract data for all grid cells at once
extracted_data_list = image_collection.map(extract_data_for_all_cells).toList(image_collection.size())

# Function to process the extracted data and concatenate arrays for each property
def extract_and_concatenate_from_featurecollection(fc):
    fc_dict = fc.getInfo()
    
    features = fc_dict['features']
    property_names = features[0]['properties'].keys()

    # Create a list of lists to hold property values for each feature
    all_property_values = {prop: [] for prop in property_names}

    for feature in features:
        for prop in property_names:
            # Only append values if they are not NaN
            if isinstance(feature['properties'][prop], list):
                all_property_values[prop].append(feature['properties'][prop])
            else:
                all_property_values[prop].append([feature['properties'][prop]])

    # Now concatenate arrays while ensuring they are of the same length
    concatenated_data = {}
    for prop in property_names:
        # Convert to NumPy array and filter out missing values
        valid_data = np.concatenate(all_property_values[prop])
        # Filter out NaN values, if any
        valid_data = valid_data[~np.isnan(valid_data)]
        concatenated_data[prop] = valid_data

    return concatenated_data

# Initialize a dictionary to hold the final concatenated data across all FeatureCollections, including 'month'
final_concatenated_data = {}
years_list = []
months_list = []

# Iterate over each FeatureCollection in the extracted data list
for i in range(image_collection.size().getInfo()):
    # Get the current image
    image = ee.Image(image_collection.toList(image_collection.size()).get(i))
    print(f'Extracting image: {i + 1}')

    # Extract the 'year' and 'month' properties from the image
    year = image.get('year').getInfo()
    month = image.get('month').getInfo()
    
    # Get the current FeatureCollection
    fc = ee.FeatureCollection(extracted_data_list.get(i))

    # Extract and concatenate the data for this FeatureCollection
    concatenated_data = extract_and_concatenate_from_featurecollection(fc)

    # Update the final concatenated data and store the 'year' and 'month' value for each image
    years_list.append(years)
    months_list.append(month)
    
    for prop, array in concatenated_data.items():
        if prop not in final_concatenated_data:
            final_concatenated_data[prop] = array
        else:
            final_concatenated_data[prop] = np.concatenate((final_concatenated_data[prop], array))

final_concatenated_data

In [ ]:
def find_array_lengths(data_dict):
    array_lengths = {}
    
    for key, value in data_dict.items():
        if hasattr(value, '__len__'):  # Check if the value has a length (e.g., an array)
            array_lengths[key] = len(value)
    
    return array_lengths

array_lengths = find_array_lengths(final_concatenated_data)
array_lengths

In [ ]:
# Create a DataFrame from the dictionary
df = pd.DataFrame.from_dict(final_concatenated_data)
df = df[['build_up_index', 'drought_code', 'duff_moisture_code', 'fine_fuel_moisture_code', 'fire_weather_index', 'initial_fire_spread_index', 'latitude', 'longitude']]
df.replace(-9999, np.nan, inplace=True)
df['month'] = np.repeat(months_list, len(df) // len(months_list))
years_flat = [year for sublist in years_list for year in sublist] # Flatten years_list
df['year'] = np.repeat(years_flat, len(df) // len(years_flat))
df.set_index(['year', 'month'], inplace=True)

df

In [ ]:
# Export single year to CSV
# Specify the CSV file path
csv_file_path = 'cems_2014.csv' # <-- Edit as necessary
df_reset = df.reset_index()

# Export the DataFrame to CSV
df_reset.to_csv(csv_file_path, index=False)

### Combine all individual years into one file

In [ ]:
# Combine all individual years to one Dataframe
file_path_pattern = "/home/users/clelland/Model/NASA_CEMS_comparison/CEMS_annual/cems_*.csv" # <-- Edit as necessary

# Get a list of all the CSV files matching the pattern
csv_files = glob.glob(file_path_pattern)

# Read and concatenate all the CSV files into one DataFrame
combined_df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
combined_df

In [ ]:
# Export to CSV
# Specify the CSV file path
csv_file_path = 'cems_0114.csv' # <-- Edit as necessary

# Export the DataFrame to CSV
combined_df.to_csv(csv_file_path, index=False)

## Process ACCESS NASA to CSV

In [ ]:
# Import downscaled FWI into an image collection
# Run and save single years before combining into one master file
years = list(range(2014, 2015))  # 2001 to 2014
months = list(range(1, 13))  # 1 to 12

# Define the new band names
band_names = ['BUI', 'DC', 'DMC', 'FFMC', 'FWI', 'FWI_N15', 'FWI_N30', 'FWI_N45', 'FWI_NP95', 'FWI_Nmid', 'ISI']

# Google Cloud Storage bucket path template
gcs_template = 'gs://clelland_fire_ml/FWI_files/ACCESS-CM2_COG/Historical/ACCESS-CM2_historical_{year}_{month}_cog.tif' # <-- Permission needed: ask Trevor

# Function to generate an ee.Image from a GCS path
def create_image(year, month):
    file_path = gcs_template.format(year=year, month=month)
    image = ee.Image.loadGeoTIFF(file_path).clip(final)

    # Get the last day of the month
    _, last_day = calendar.monthrange(year, month)
    
    # Define the start and end dates
    start_date = f'{year}-{month:02d}-01'
    end_date = f'{year}-{month:02d}-{last_day:02d}'

    # Rename the bands of the image
    image = image.rename(band_names)

    # Handle missing pixels for BUI and FWI bands
    bui = image.select('BUI')
    fwi = image.select('FWI')

    # Apply unmask only where pixels are missing
    bui_filled = bui.unmask(-9999, sameFootprint=True).updateMask(image.select('FFMC'))
    fwi_filled = fwi.unmask(-9999, sameFootprint=True).updateMask(image.select('FFMC'))

    # Replace the original BUI and FWI with the filled versions (preserve existing valid pixels)
    coords = ee.Image.pixelLonLat().clip(final).updateMask(image.select('FFMC'))
    image = image.addBands([bui_filled.rename('BUI'), fwi_filled.rename('FWI')], overwrite=True).addBands(coords)

    # Set properties including the start and end dates
    image = image.set({
        'scenario': 'historical',
        'year': year,
        'month': month,
        'system:time_start': ee.Date(start_date).millis(),
        'system:time_end': ee.Date(end_date).millis()
    })
    return image

# Create an empty list to hold the images
image_list = []

# Loop through scenarios, years, and months, and import the files
for year in years:
    for month in months:
        try:
            image = create_image(year, month)
            image_list.append(image)
        except Exception as e:
            print(f"Error loading image for {year}-{month}: {e}")

# Convert the list of images to an Earth Engine ImageCollection
access_fwi_images = ee.ImageCollection(image_list)

# Print the image collection size to confirm
print("Image collection created with", access_fwi_images.size().getInfo(), "images")

In [ ]:
image_collection = access_fwi_images

# Function to extract data for all grid cells from each image in the ImageCollection
def extract_data_for_all_cells(image):
    # Use reduceRegions to process all grid cells at once, instead of iterating over them
    data = image.reduceRegions(
        collection=quarter_degree_grid,
        reducer=ee.Reducer.toList(),
        scale=27829.87269831839,
        crs='EPSG:4326'
    )
    return data

# Iterate over images and extract data for all grid cells at once
extracted_data_list = image_collection.map(extract_data_for_all_cells).toList(image_collection.size())

# Function to process the extracted data and concatenate arrays for each property
def extract_and_concatenate_from_featurecollection(fc):
    fc_dict = fc.getInfo()
    
    features = fc_dict['features']
    property_names = features[0]['properties'].keys()

    concatenated_data = {prop: [] for prop in property_names}

    for feature in features:
        for prop in property_names:
            concatenated_data[prop].extend(feature['properties'][prop])

    # Convert lists to NumPy arrays
    for prop in concatenated_data:
        concatenated_data[prop] = np.array(concatenated_data[prop])

    return concatenated_data

# Initialize a dictionary to hold the final concatenated data across all FeatureCollections, including 'month'
final_concatenated_data = {}
years_list = []
months_list = []

# Iterate over each FeatureCollection in the extracted data list
for i in range(image_collection.size().getInfo()):
    # Get the current image
    image = ee.Image(image_collection.toList(image_collection.size()).get(i))
    print(f'Extracting image: {i + 1}')

    # Extract the 'year' and 'month' properties from the image
    year = image.get('year').getInfo()
    month = image.get('month').getInfo()
    
    # Get the current FeatureCollection
    fc = ee.FeatureCollection(extracted_data_list.get(i))

    # Extract and concatenate the data for this FeatureCollection
    concatenated_data = extract_and_concatenate_from_featurecollection(fc)

    # Update the final concatenated data and store the 'year' and 'month' value for each image
    years_list.append(years)
    months_list.append(month)
    
    for prop, array in concatenated_data.items():
        if prop not in final_concatenated_data:
            final_concatenated_data[prop] = array
        else:
            final_concatenated_data[prop] = np.concatenate((final_concatenated_data[prop], array))

final_concatenated_data

In [ ]:
def find_array_lengths(data_dict):
    array_lengths = {}
    
    for key, value in data_dict.items():
        if hasattr(value, '__len__'):  # Check if the value has a length (e.g., an array)
            array_lengths[key] = len(value)
    
    return array_lengths

array_lengths = find_array_lengths(final_concatenated_data)
array_lengths

In [ ]:
# Create a DataFrame from the dictionary
df = pd.DataFrame.from_dict(final_concatenated_data)
df = df[['BUI', 'DC', 'DMC', 'FFMC', 'FWI', 'ISI', 'latitude', 'longitude']]
df.replace(-9999, np.nan, inplace=True)
years_flat = [year for sublist in years_list for year in sublist] # Flatten years_list
df['year'] = np.repeat(years_flat, len(df) // len(years_flat))
df['month'] = np.repeat(months_list, len(df) // len(months_list))
df.set_index(['year', 'month'], inplace=True)

df

In [ ]:
# Export to CSV
# Specify the CSV file path
csv_file_path = 'access_2014.csv'
df_reset = df.reset_index()

# Export the DataFrame to CSV
df_reset.to_csv(csv_file_path, index=False)

### Combine all individual years into one file

In [ ]:
# Combine all individual years to one Dataframe
file_path_pattern = "/home/users/clelland/Model/NASA_CEMS_comparison/ACCESS_annual/access_*.csv" # <-- Edit as necessary

# Get a list of all the CSV files matching the pattern
csv_files = glob.glob(file_path_pattern)

# Read and concatenate all the CSV files into one DataFrame
combined_df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
combined_df

In [ ]:
# Export to CSV
# Specify the CSV file path
csv_file_path = 'access_0114.csv' # <-- Edit as necessary

# Export the DataFrame to CSV
combined_df.to_csv(csv_file_path, index=False)

## Process MRI NASA to CSV

In [ ]:
# Import downscaled FWI into an image collection
# Run and save single years before combining into one master file
years = list(range(2014, 2015))  # 2001 to 2014
months = list(range(1, 13))  # 1 to 12

# Define the new band names
band_names = ['BUI', 'DC', 'DMC', 'FFMC', 'FWI', 'FWI_N15', 'FWI_N30', 'FWI_N45', 'FWI_NP95', 'FWI_Nmid', 'ISI']

# Google Cloud Storage bucket path template
gcs_template = 'gs://clelland_fire_ml/FWI_files/MRI-ESM2-0_COG/Historical/MRI-ESM2-0_historical_{year}_{month}_cog.tif' # Permission required: ask Trevor

# Function to generate an ee.Image from a GCS path
def create_image(year, month):
    file_path = gcs_template.format(year=year, month=month)
    image = ee.Image.loadGeoTIFF(file_path).clip(final)

    # Get the last day of the month
    _, last_day = calendar.monthrange(year, month)
    
    # Define the start and end dates
    start_date = f'{year}-{month:02d}-01'
    end_date = f'{year}-{month:02d}-{last_day:02d}'

    # Rename the bands of the image
    image = image.rename(band_names)

    # Handle missing pixels for BUI and FWI bands
    bui = image.select('BUI')
    fwi = image.select('FWI')

    # Apply unmask only where pixels are missing
    bui_filled = bui.unmask(-9999, sameFootprint=True).updateMask(image.select('FFMC'))
    fwi_filled = fwi.unmask(-9999, sameFootprint=True).updateMask(image.select('FFMC'))

    # Replace the original BUI and FWI with the filled versions (preserve existing valid pixels)
    coords = ee.Image.pixelLonLat().clip(final).updateMask(image.select('FFMC'))
    image = image.addBands([bui_filled.rename('BUI'), fwi_filled.rename('FWI')], overwrite=True).addBands(coords)

    # Set properties including the start and end dates
    image = image.set({
        'scenario': 'historical',
        'year': year,
        'month': month,
        'system:time_start': ee.Date(start_date).millis(),
        'system:time_end': ee.Date(end_date).millis()
    })
    return image

# Create an empty list to hold the images
image_list = []

# Loop through scenarios, years, and months, and import the files
for year in years:
    for month in months:
        try:
            image = create_image(year, month)
            image_list.append(image)
        except Exception as e:
            print(f"Error loading image for {year}-{month}: {e}")

# Convert the list of images to an Earth Engine ImageCollection
mri_fwi_images = ee.ImageCollection(image_list)

# Print the image collection size to confirm
print("Image collection created with", mri_fwi_images.size().getInfo(), "images")

In [ ]:
image_collection = mri_fwi_images

# Function to extract data for all grid cells from each image in the ImageCollection
def extract_data_for_all_cells(image):
    # Use reduceRegions to process all grid cells at once, instead of iterating over them
    data = image.reduceRegions(
        collection=quarter_degree_grid,
        reducer=ee.Reducer.toList(),
        scale=27829.87269831839,
        crs='EPSG:4326'
    )
    return data

# Iterate over images and extract data for all grid cells at once
extracted_data_list = image_collection.map(extract_data_for_all_cells).toList(image_collection.size())

# Function to process the extracted data and concatenate arrays for each property
def extract_and_concatenate_from_featurecollection(fc):
    fc_dict = fc.getInfo()
    
    features = fc_dict['features']
    property_names = features[0]['properties'].keys()

    concatenated_data = {prop: [] for prop in property_names}

    for feature in features:
        for prop in property_names:
            concatenated_data[prop].extend(feature['properties'][prop])

    # Convert lists to NumPy arrays
    for prop in concatenated_data:
        concatenated_data[prop] = np.array(concatenated_data[prop])

    return concatenated_data

# Initialize a dictionary to hold the final concatenated data across all FeatureCollections, including 'month'
final_concatenated_data = {}
years_list = []
months_list = []

# Iterate over each FeatureCollection in the extracted data list
for i in range(image_collection.size().getInfo()):
    # Get the current image
    image = ee.Image(image_collection.toList(image_collection.size()).get(i))
    print(f'Extracting image: {i + 1}')
    
    # Extract the 'year' and 'month' properties from the image
    year = image.get('year').getInfo()
    month = image.get('month').getInfo()
    
    # Get the current FeatureCollection
    fc = ee.FeatureCollection(extracted_data_list.get(i))

    # Extract and concatenate the data for this FeatureCollection
    concatenated_data = extract_and_concatenate_from_featurecollection(fc)

    # Update the final concatenated data and store the 'year' and 'month' value for each image
    years_list.append(years)
    months_list.append(month)
    
    for prop, array in concatenated_data.items():
        if prop not in final_concatenated_data:
            final_concatenated_data[prop] = array
        else:
            final_concatenated_data[prop] = np.concatenate((final_concatenated_data[prop], array))

final_concatenated_data

In [ ]:
def find_array_lengths(data_dict):
    array_lengths = {}
    
    for key, value in data_dict.items():
        if hasattr(value, '__len__'):  # Check if the value has a length (e.g., an array)
            array_lengths[key] = len(value)
    
    return array_lengths

array_lengths = find_array_lengths(final_concatenated_data)
array_lengths

In [ ]:
# Create a DataFrame from the dictionary
df = pd.DataFrame.from_dict(final_concatenated_data)
df = df[['BUI', 'DC', 'DMC', 'FFMC', 'FWI', 'ISI', 'latitude', 'longitude']]
df.replace(-9999, np.nan, inplace=True)
years_flat = [year for sublist in years_list for year in sublist] # Flatten years_list
df['year'] = np.repeat(years_flat, len(df) // len(years_flat))
df['month'] = np.repeat(months_list, len(df) // len(months_list))
df.set_index(['year', 'month'], inplace=True)

df

In [ ]:
# Export to CSV
# Specify the CSV file path
csv_file_path = 'mri_2014.csv' # <-- Edit as necessary
df_reset = df.reset_index()

# Export the DataFrame to CSV
df_reset.to_csv(csv_file_path, index=False)

### Combine all individual years into one file

In [ ]:
# Combine all individual years to one Dataframe
file_path_pattern = "/home/users/clelland/Model/NASA_CEMS_comparison/MRI_annual/mri_*.csv" # <-- Edit as necessary

# Get a list of all the CSV files matching the pattern
csv_files = glob.glob(file_path_pattern)

# Read and concatenate all the CSV files into one DataFrame
combined_df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
combined_df

In [ ]:
# Export to CSV
# Specify the CSV file path
csv_file_path = 'mri_0114.csv' # <-- Edit as necessary

# Export the DataFrame to CSV
combined_df.to_csv(csv_file_path, index=False)